# Nov 10, 2025: fit modularity maximization

conda env: gt

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import dill as pickle 
from scipy import sparse, stats
from scipy.special import gammaln
import graph_tool.all as gt
from glob import glob
import re 
import bct
from tqdm import tqdm

from nilearn import image, plotting
from sklearn.preprocessing import OneHotEncoder

# plotting
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cycler import cycler
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

import colorcet as cc 

import seaborn as sns
# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS(): pass

args = ARGS()

args.SEED = 100
np.random.seed(args.SEED)

In [3]:
args.ANALYSIS = 'trial-end'
# args.WINDOW = np.arange(5, 8+1)
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson'
args.THRESHOLD = f'signed'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'grp'

BASE_path = f'/home/govindas/lab-data/aba'
PARC_DESC = f'NEWMAX_ROIs_final_gm_104_2mm' #f'ABA_ROIs_final_gm_36'
ROI_path = (
    f'{BASE_path}/{PARC_DESC}'
)
os.makedirs(ROI_path, exist_ok=True)
TS_path = f'{ROI_path}/roi-timeseries'
os.makedirs(TS_path, exist_ok=True)

ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/analysis-{args.ANALYSIS}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLD}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
SBM_path = f'{ROI_RESULTS_path}/model-fits'
DIAG_path = f'{ROI_RESULTS_path}/diagnostics_insets'
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.makedirs(ESTIM_path, exist_ok=True)

In [4]:
args.dc, args.sbm = True, 'm'
args.nested = args.sbm == 'h'

def sbm_name(args):
    dc = f'dc' if args.dc else f'nd'
    dc = f'' if args.sbm in ['a', 'm'] else dc
    file = f'sbm-{dc}-{args.sbm}'
    return file

SBM = sbm_name(args)
SBM

'sbm--m'

In [5]:
gfiles = sorted(glob(f'{GRAPH_path}/*'))
gfiles

['/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_104_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-grp/graphs/cond-PLAY_highR_desc-graph.gt.gz',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_104_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-grp/graphs/cond-PLAY_highT_desc-graph.gt.gz',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_104_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-grp/graphs/cond-PLAY_lowR_desc-graph.gt.gz',
 '/home/govindas/lab-data/aba/NEWMAX_ROIs_final_gm_104_2mm/analysis-trial-end/graph-constructed/method-pearson/threshold-signed/edge-binary/density-20/layer-individual/unit-grp/graphs/cond-PLAY_lowT_desc-graph.gt.gz']

In [6]:
def adjacency_matrix(g):
    A = np.zeros((g.num_vertices(), g.num_vertices()), dtype=int)
    edges = [list(edge) for edge in list(g.get_edges())]
    v1, v2 = list(zip(*edges))
    weights = list(g.ep['weight'])
    A[v1, v2] = weights
    A = (A + A.T)
    return A


In [7]:
def get_pi(b):
    b = b.reshape(-1, 1)
    cats = [np.arange(np.max(b)+1)]
    enc = OneHotEncoder(
        sparse_output=False,
        categories=cats,
    )
    pi = enc.fit_transform(b)
    return pi

In [8]:
args.resolution = 0.5
args.repetitions = 1000

In [9]:
df = []
for gfile in tqdm(gfiles):

    match = re.search(r"cond-([^_]+)_([^_]+)_desc-", gfile)
    if match:
        cond, name = match.groups()
        cond = '_'.join([cond, name]) 
    
    A = adjacency_matrix(gt.load_graph(gfile))
    
    cpartition = bct.consensus_und(
        A, 
        tau=args.resolution,
        reps=args.repetitions,
    )
    pi = get_pi(cpartition)
    
    row = pd.DataFrame(dict(
        cond=[cond],
        graph=[A],
        pi=[pi],
    ))
    df += [row]
    # break
df = pd.concat(df).reset_index(drop=True)

100%|██████████| 4/4 [02:32<00:00, 38.20s/it]


In [10]:
df

,cond,graph,pi
0,PLAY_highR,"[[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,PLAY_highT,"[[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,PLAY_lowR,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,PLAY_lowT,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,...","[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [11]:
for idx, row in df.iterrows():
    cond = row['cond']
    out_folder = f'{ESTIM_path}/{SBM}/cond-{cond}/membership-mats-group-aligned'
    os.makedirs(out_folder, exist_ok=True)
    
    with open(f'{out_folder}/desc-mats.pkl', 'wb') as f:
        pickle.dump(row['pi'], f)

In [12]:
PROJ_path = f'/home/govindas/mounts/kaba/aba'
DATA_path = f'{PROJ_path}/dataset/first_level/voxelwise_13TR_4plays_offset_reducedRuns_trialwise'
PARC_path = f'{PROJ_path}/ROI_mask'
PARC_DESC = f'NEWMAX_ROIs_final_gm_104_2mm' #f'ABA_ROIs_final_gm_36'
PARC_file = f'{PARC_path}/{PARC_DESC}.nii.gz'
PARC_img = image.load_img(PARC_file)
PARC_data = PARC_img.get_fdata()

In [13]:
def get_stat_image(b):
    b_data = np.zeros_like(PARC_data)
    roi_labels = np.unique(PARC_data)[1:] # skip background 0
    for idx, label in enumerate(roi_labels):
        b_data[PARC_data == label] = b[idx]
    b_img = image.new_img_like(PARC_img, b_data)
    return b_img

In [16]:
for idx, row in df.iterrows():
    cond = row['cond']
    out_folder = f'{ESTIM_path}/{SBM}/cond-{cond}/membership-mats-group-aligned/marginal-visuals/2d'
    os.makedirs(out_folder, exist_ok=True)
    
    pi = row['pi']
    for idx_col in tqdm(range(pi.shape[-1])):
        b_img = get_stat_image(pi[:, idx_col])
        plotting.plot_stat_map(
            stat_map_img=b_img,
            cmap=cc.cm.CET_L12,
            display_mode='z',
            draw_cross=False,
            annotate=False,
            threshold=0.0,
            output_file=f'{out_folder}/comm-{idx_col:02d}.svg',
        )
    
    # break

100%|██████████| 19/19 [01:47<00:00,  5.65s/it]
